In [167]:
def parse_input(data):
    lines = data.strip().split('\n')
    index = 0
    stocks = {}
    accounts = {}
    eligible_accounts = defaultdict(list)
    eligible_flows = defaultdict(list)
    balances = defaultdict(dict)

    # Read number of stocks and their prices
    n = int(lines[index].strip())
    index += 1
    for _ in range(n):
        stock_id, price = lines[index].strip().split(',')
        stocks[stock_id] = float(price)
        index += 1

    # Read number of accounts
    n = int(lines[index].strip())
    index += 1
    for _ in range(n):
        account_id, account_type, parent_account = lines[index].strip().split(',')
        accounts[account_id] = (account_type, parent_account)
        index += 1

    # Read eligible accounts for each stock
    n = int(lines[index].strip())
    index += 1
    for _ in range(n):
        stock_id, account_id = lines[index].strip().split(',')
        eligible_accounts[stock_id].append(account_id)
        index += 1

    # Read eligible flows between accounts
    n = int(lines[index].strip())
    index += 1
    for _ in range(n):
        stock_id, source_id, dest_id = lines[index].strip().split(',')
        eligible_flows[stock_id].append((source_id, dest_id))
        index += 1
        


    # Read balances for each stock in each account
    n = int(lines[index].strip())
    index += 1
    for _ in range(n):
        stock_id, account_id, quantity = lines[index].strip().split(',')
        balances[stock_id][account_id] = int(quantity)
        index += 1

    return stocks, accounts, eligible_accounts, eligible_flows, balances

In [168]:
input_data = """2
P1,2.5
P2,1.25
3
Loc1,CUSTODY,1
Loc2,CUSTODY,2
Loc3,TRIPARTY,2
6
P1,Loc1
P1,Loc2
P1,Loc3
P2,Loc1
P2,Loc2
P2,Loc3
7
P1,Loc1,Loc2
P1,Loc1,Loc3
P1,Loc2,Loc1
P1,Loc2,Loc3
P2,Loc1,Loc2
P2,Loc2,Loc3
P2,Loc3,Loc1
3
P1,Loc1,10
P1,Loc2,-5
P2,Loc1,5"""


In [169]:
parse_input(input_data)

({'P1': 2.5, 'P2': 1.25},
 {'Loc1': ('CUSTODY', '1'),
  'Loc2': ('CUSTODY', '2'),
  'Loc3': ('TRIPARTY', '2')},
 defaultdict(list,
             {'P1': ['Loc1', 'Loc2', 'Loc3'], 'P2': ['Loc1', 'Loc2', 'Loc3']}),
 defaultdict(list,
             {'P1': [('Loc1', 'Loc2'),
               ('Loc1', 'Loc3'),
               ('Loc2', 'Loc1'),
               ('Loc2', 'Loc3')],
              'P2': [('Loc1', 'Loc2'), ('Loc2', 'Loc3'), ('Loc3', 'Loc1')]}),
 defaultdict(dict, {'P1': {'Loc1': 10, 'Loc2': -5}, 'P2': {'Loc1': 5}}))

In [170]:
from collections import defaultdict, deque
import heapq

def find_minimum_transactions(stocks, accounts, eligible_flows, balances):
    # Step 1: Identify negative and positive balances
    negative_balances = defaultdict(dict)  # stock -> {account_id: quantity}
    positive_balances = defaultdict(dict)  # stock -> {account_id: quantity}

    for stock_id, account_data in balances.items():
        for account_id, quantity in account_data.items():
            if quantity < 0:
                negative_balances[stock_id][account_id] = quantity
            elif quantity > 0:
                positive_balances[stock_id][account_id] = quantity

    # Step 2: Build a graph for eligible flows
    graph = defaultdict(list)
    for stock_id, flows in eligible_flows.items():
        for source_id, dest_id in flows:
            graph[source_id].append(dest_id)

    # Step 3: Function to perform Dijkstra's algorithm to find shortest paths
    def dijkstra(source, stock):
        min_distances = {account_id: float('inf') for account_id in accounts}
        min_distances[source] = 0
        priority_queue = [(0, source)]  # (distance, account_id)

        while priority_queue:
            current_distance, current_account = heapq.heappop(priority_queue)

            if current_distance > min_distances[current_account]:
                continue

            for neighbor in graph[current_account]:
                if stock in eligible_accounts and neighbor in eligible_accounts[stock]:
                    distance = current_distance + 1  # Assume cost is 1 for each transaction

                    if distance < min_distances[neighbor]:
                        min_distances[neighbor] = distance
                        heapq.heappush(priority_queue, (distance, neighbor))

        return min_distances

    # Step 4: Group accounts by parent account
    accounts_by_parent = defaultdict(list)
    for account_id, (_, parent_account) in accounts.items():
        accounts_by_parent[parent_account].append(account_id)

    # Step 5: Find transactions based on the shortest paths
    transactions = []
    for stock_id, neg_accounts in negative_balances.items():
        for neg_account, neg_quantity in neg_accounts.items():
            required = -neg_quantity  # Quantity needed to fulfill
            min_distances = dijkstra(neg_account, stock_id)

            # Step 6: Prioritize transactions within the same parent account
            parent_account = accounts[neg_account][1]
            sibling_accounts = accounts_by_parent[parent_account]

            # Gather sibling positive balances for the current stock
            sibling_positive_balances = {acc: pos_qty for acc, pos_qty in positive_balances[stock_id].items() if acc in sibling_accounts}

            # Select the account for transaction
            for pos_account, pos_quantity in sibling_positive_balances.items():
                if pos_quantity > 0 and min_distances[pos_account] < float('inf'):
                    if required <= 0:
                        break
                    
                    transfer_quantity = min(required, pos_quantity)
                    transactions.append((stock_id, pos_account, neg_account, transfer_quantity))
                    required -= transfer_quantity

            # If required quantity is still not fulfilled, continue with other positive balances
            if required > 0:
                for pos_account, pos_quantity in positive_balances[stock_id].items():
                    if pos_quantity > 0 and min_distances[pos_account] < float('inf'):
                        transfer_quantity = min(required, pos_quantity)
                        transactions.append((stock_id, pos_account, neg_account, transfer_quantity))
                        required -= transfer_quantity
                        if required <= 0:
                            break

    return transactions



stocks, accounts, eligible_accounts, eligible_flows, balances = parse_input(input_data)

transactions = find_minimum_transactions(stocks, accounts, eligible_flows, balances)

# Output the transactions in sorted order
transactions.sort(key=lambda x: (x[0], x[1], x[2]))

for transaction in transactions:
    print(f"{transaction[0]},{transaction[1]},{transaction[2]},{transaction[3]}")


P1,Loc1,Loc2,5


In [171]:
def find_non_zero_balances(balances, transactions):
    # Step 1: Create a copy of balances to update
    updated_balances = defaultdict(dict)

    # Initialize updated balances with the existing balances
    for stock_id, account_data in balances.items():
        for account_id, quantity in account_data.items():
            updated_balances[stock_id][account_id] = quantity

    # Step 2: Update balances based on the transactions
    for stock_id, source_account, destination_account, quantity in transactions:
        # Deduct from the source account and add to the destination account
        updated_balances[stock_id][source_account] -= quantity
        updated_balances[stock_id][destination_account] += quantity

    # Step 3: Identify accounts with non-zero balances
    non_zero_balances = defaultdict(dict)
    for stock_id, account_data in updated_balances.items():
        for account_id, quantity in account_data.items():
            if quantity != 0:
                non_zero_balances[stock_id][account_id] = quantity

    return non_zero_balances


non_zero_balances = find_non_zero_balances(balances, transactions)


In [176]:
from collections import deque, defaultdict

def bfs_find_shortest_path(stock_id, start_account, pathways, triparty_accounts, balances):
    if stock_id not in balances or start_account not in balances[stock_id]:
        return None  # No valid starting point

    queue = deque([(start_account, balances[stock_id][start_account], [start_account])])  # (current account, current quantity, path)
    visited = set()  # To keep track of visited accounts

    while queue:
        current_account, current_quantity, path = queue.popleft()

        # If the current account is a triparty account, return the path
        if current_account in triparty_accounts:
            if len(path) > 1:
                # Generate intermediate pathways if distance is greater than 1
                pathways_output = []
                for i in range(len(path) - 1):
                    source = path[i]
                    dest = path[i + 1]
                    pathways_output.append((stock_id, source, dest, current_quantity))
                return pathways_output  # Return the generated pathways

            return [(stock_id, start_account, current_account, current_quantity)]  # Direct path for distance 1

        visited.add(current_account)

        # Check eligible pathways for the current stock
        for source, destination in pathways[stock_id]:
            if source == current_account and destination not in visited:
                # Deduct from the source and add to the destination account
                if current_quantity > 0:
                    queue.append((destination, current_quantity, path + [destination]))  # Append new path

    return None  # If no triparty account is reachable

def find_non_zero_balances(balances, transactions):
    # Step 1: Create a copy of balances to update
    updated_balances = defaultdict(dict)

    # Initialize updated balances with the existing balances
    for stock_id, account_data in balances.items():
        for account_id, quantity in account_data.items():
            updated_balances[stock_id][account_id] = quantity

    # Step 2: Update balances based on the transactions
    for stock_id, source_account, destination_account, quantity in transactions:
        # Deduct from the source account and add to the destination account
        updated_balances[stock_id][source_account] -= quantity
        updated_balances[stock_id][destination_account] += quantity

    # Step 3: Identify accounts with non-zero balances
    non_zero_balances = defaultdict(dict)
    for stock_id, account_data in updated_balances.items():
        for account_id, quantity in account_data.items():
            if quantity != 0:
                non_zero_balances[stock_id][account_id] = quantity

    return non_zero_balances

def find_shortest_paths_to_triparty(non_zero_balances, pathways, triparty_accounts):
    all_shortest_paths = []

    for stock_id, accounts in non_zero_balances.items():
        for account_id, quantity in accounts.items():
            path = bfs_find_shortest_path(stock_id, account_id, pathways, triparty_accounts, non_zero_balances)
            if path:
                all_shortest_paths.extend(path)

    return all_shortest_paths



# Define eligible flows (pathways)
pathways = defaultdict(list)
flows = []

# Convert defaultdict to the desired list format
for stock_id, flow_list in eligible_flows.items():  # Use 'flow_list' to avoid conflict with 'flows'
    for source, dest in flow_list:
        flows.append((stock_id, source, dest))

# Populate pathways
for stock_id, source, dest in flows:
    pathways[stock_id].append((source, dest))


# Find non-zero balances
non_zero_balances = find_non_zero_balances(balances, transactions)



# Define triparty accounts
triparty_accounts = {account for account, details in accounts.items() if details[0] == 'TRIPARTY'}

# Find shortest paths to triparty accounts
shortest_paths = find_shortest_paths_to_triparty(non_zero_balances, pathways, triparty_accounts)

# Output the shortest paths in the specified format
for stock_id, source_account, dest_account, quantity in shortest_paths:
    print(f"{stock_id},{source_account},{dest_account},{quantity}")


P1,Loc1,Loc3,5
P2,Loc1,Loc2,5
P2,Loc2,Loc3,5


In [180]:
def remove_larger_duplicate_paths(paths):
    unique_paths = set()
    filtered_paths = []

    for stock_id, source, dest, quantity in paths:
        path_tuple = (stock_id, source, dest, quantity)
        
        # Check if this path or its reverse is already in the set
        if (source, dest) not in unique_paths and (dest, source) not in unique_paths:
            unique_paths.add((source, dest))  # Add the path to the set
            filtered_paths.append(path_tuple)  # Keep the original path

    return filtered_paths


# Remove larger duplicate pathways
filtered_paths = remove_larger_duplicate_paths(shortest_paths)

# Output the filtered paths
for transaction in transactions:
    print(f"{transaction[0]},{transaction[1]},{transaction[2]},{transaction[3]}")
for stock_id, source, dest, quantity in filtered_paths:
    print(f"{stock_id},{source},{dest},{quantity}")


P1,Loc1,Loc2,5
P1,Loc1,Loc3,5
P2,Loc1,Loc2,5
P2,Loc2,Loc3,5


In [178]:
# Given dictionary of accounts
accounts = {
    'Loc1': ('CUSTODY', '1'),
    'Loc2': ('CUSTODY', '2'),
    'Loc3': ('TRIPARTY', '2')
}

# Extract triparty accounts into a set
triparty_accounts = {account for account, details in accounts.items() if details[0] == 'TRIPARTY'}

# Print the result
print(triparty_accounts)


{'Loc3'}


In [179]:
from collections import defaultdict

def parse_input():
    stocks = {}
    accounts = {}
    eligible_accounts = defaultdict(list)
    eligible_flows = defaultdict(list)
    balances = defaultdict(dict)

    # Read number of stocks and their prices
    n = int(input().strip())
    for _ in range(n):
        stock_id, price = input().strip().split(',')
        stocks[stock_id] = float(price)

    # Read number of accounts
    n = int(input().strip())
    for _ in range(n):
        account_id, account_type, parent_account = input().strip().split(',')
        accounts[account_id] = (account_type, parent_account)

    # Read eligible accounts for each stock
    n = int(input().strip())
    for _ in range(n):
        stock_id, account_id = input().strip().split(',')
        eligible_accounts[stock_id].append(account_id)

    # Read eligible flows between accounts
    n = int(input().strip())
    for _ in range(n):
        stock_id, source_id, dest_id = input().strip().split(',')
        eligible_flows[stock_id].append((source_id, dest_id))

    # Read balances for each stock in each account
    n = int(input().strip())
    for _ in range(n):
        stock_id, account_id, quantity = input().strip().split(',')
        balances[stock_id][account_id] = int(quantity)

    return stocks, accounts, eligible_accounts, eligible_flows, balances


In [181]:
combined_results = transactions + filtered_paths

# Sort the combined results
sorted_results = sorted(combined_results, key=lambda x: (x[0], x[1], x[2], x[3]))

# Output the sorted results
for transaction in sorted_results:
    print(f"{transaction[0]},{transaction[1]},{transaction[2]},{transaction[3]}")

P1,Loc1,Loc2,5
P1,Loc1,Loc3,5
P2,Loc1,Loc2,5
P2,Loc2,Loc3,5


In [182]:
import pandas as pd

# Load the data from the CSV file
filename = input() # Read the input filename
df = pd.read_csv(filename)

# Extract rows where seconds_bucket is 31200
extracted_df = df[df['seconds_bucket'] == 31200]
extracted_df.reset_index(drop=True, inplace=True)

# Calculate average close_volume for every 50 rows
for i in range(0, len(extracted_df), 50):
    group = extracted_df.iloc[i:i+50]
    if not group.empty:
        symbol_id = group['symbol_id'].iloc[0]  # Get the symbol_id for the group
        avg_close_volume = group['close_volume'].mean()  # Calculate the average
        print(f"{symbol_id}, {avg_close_volume:.1f}")  # Print in the specified format

C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


sample2 (1).csv
1029677210, 3184360.9
1077784782, 630194.0
1084320043, 8299645.1
1106122882, 217742.4
